In [ ]:
optimizer: str = 'rmsprop'
epochs: int = 35
batch_size: int = 32

In [ ]:
# Parameters
optimizer = "adam"
epochs = 20
batch_size = 32


In [ ]:
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes=3)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=3)

# Size of dataset
size = iris.data.shape[0]

In [ ]:
# Build the neural network model
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(4,)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

In [ ]:
# Compile the model
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
from codecarbon import EmissionsTracker
from examples.serializer import Serializer

# Serialize the model
serializer = Serializer()
row = serializer.serialize(model, epochs, batch_size, size)

# Start the emissions tracker
tracker = EmissionsTracker(log_level='critical')
tracker.start()

# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)

# Save the model carbon emissions
row['co2'] = tracker.stop()

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

In [ ]:
from csv import DictWriter

# Save the model metadata
with open('results.csv', 'a') as f:
    writer = DictWriter(f, fieldnames=row.keys())
    writer.writerow(row)